<a href="https://colab.research.google.com/github/shmanth/Ad_Placement_Optimization_Using_Reinforcement_learning/blob/master/Ad_Placement_Optimization_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Predicting the ad positioning using Q-Learning





Placement of ads on website is the primary problem for companies that operate on ad revenue. The position where the ad is placed plays pivotal role on whether or not the ad will be clicked. Here we have the following choices:

Place them randomly, or
Place the ad on the same position
The problem with placing the ad on the same position is the user, after a certain time, will start ignoring the space since he's used to seeing ad at the place, he will end up ignoring that particular position hereafter. Hence, this will reduce the number of clicks on ads. The problem with the former option, placing them randomly, is it wouldn't take optimal positions into consideration. For instance, text beside images are viewed higher number of times than those text which are placed at a distance. It is infeasible to go through every website and repeat the procedure.

Solution: Reinforcement Learning

Using Reinforcement Learning we can approximate the human behavior.

Why Reinforcement Learning?
We cannot use traditional Machine Learning here, since it requires:

Huge data
Features
Tuning of many hyperparameters
And we neither have huge data, nor features. The only data we have is the position of the banner/ad and whether or not it was clicked. We will use this dataset from Kaggle: https://www.kaggle.com/akram24/ads-ctr-optimisation. We will solve this problem using both model-based (Policy iteration) and model-free methods(Q-Learning & Monte-Carlo). We'll simplify some assumptions for model-based technique.

Notebook Layout
MDP environment - Understanding the dataset
Random Policy - placing the ads randomly on different webpages
Max policy - placing the ad where it is clicked maximum number of times
Model-based Method - Policy Iteration
Model-free Method - Q-learning & Monte Carlo

In [29]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import time

Dataset
Our environment will be the dataset. It contains 10 ads position per row having values either 1, when the ad is clicked, or 0 when it is not. Every row can be considered as a state in the space, considering it kind of a navigation across multiple pages (on website, for instance) Lets load the dataset and visualize the first few rows.

state = webpage
action = placing the ad at any of the 10 positions on a webpage
reward = +1 if the ad was clicked at the position; else 0
Transition Probability: next webpage that user will end up in is random; therefore, it is 1/(total_webpages -1)

In [30]:
env_data= pd.read_csv("/content/drive/MyDrive/Data_colab_practice_august_2023/Ads_CTR_Optimisation.csv")
env_data.head()

,Ad 1,Ad 2,Ad 3,Ad 4,Ad 5,Ad 6,Ad 7,Ad 8,Ad 9,Ad 10
0,1,0,0,0,1,0,0,0,1,0
1,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0


In [31]:
env_data.describe()

,Ad 1,Ad 2,Ad 3,Ad 4,Ad 5,Ad 6,Ad 7,Ad 8,Ad 9,Ad 10
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000
mean,0.170300,0.129500,0.072800,0.119600,0.269500,0.012600,0.111200,0.209100,0.095200,0.04890
std,0.375915,0.335769,0.259821,0.324509,0.443722,0.111546,0.314395,0.406686,0.293506,0.21567
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
75%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000


In [32]:
env_data["Ad 1"].value_counts()

0    8297
1    1703
Name: Ad 1, dtype: int64

### Random policy -
If we were to not have Reinforcement Learning, we would place the ads randomly at given positions. We will now simulate the same



In [33]:
#Choosing a random position to place the ad
reward= 0 # initializing the reward

for i in range(len(env_data)):
    action= np.random.randint(0,10)
    # check if random action matches with the ad position
    if env_data.values[i][action] == 1:
      # Increase the rewards if the position matches
      reward += 1
print("Total reward: {}".format(reward))

Total reward: 1219


The total rewards is too low as compared to the data


### Using Max Policy
Another question we might ask: is it right to display the ad where it is clicked the most number of times. For instance, there might be a certain position where the ad clicked with a higher probability. Since the values of the rows is either 1 or 0, we can sum across the columns and count the number of times ad in the position was clicked.

In [34]:
click_counts= env_data.sum(axis=0)
counts= pd.DataFrame({"ad": np.arange(1,11), "counts": click_counts})
counts.set_index("ad")

,counts
ad,
1,1703
2,1295
3,728
4,1196
5,2695
6,126
7,1112
8,2091
9,952


Which indicates ad 5 was clicked 2695 times. So if we were to always place an ad on position 5, it would be click around 2695 times. But can we do better?

## Dynamic Programming - Policy iteration

In [35]:
state_size = len(env_data)

state_list = []
for state in range(state_size):
    state_list.append(state)

In [ ]:
start= time.time()

# Action can be placing the at any of the ten positions.
action= np.arange(0,10)
policy= [np.random.choice(action) for x in range(state_size)]
# Taking random action for the first time
first_time= True
#Delta
small_change= 1e-20
#Discount Factor
gamma= 0.9

episodes= 0
max_episodes= 500

V= dict()
# Setting terminal state reward to 1
V[10000]= 1
for i in range(state_size):
  V[i]= np.random.random()

deltas=[]

while episodes < max_episodes:
  while True:
    if episodes > max_episodes:
      break
    episodes += 1
    if episodes % 100 == 0:
      print("Current episode is {}".format(episodes) )
    biggest_change= 0
    for state in range(state_size):
      old_V= V[state]
      #Taking random action according to policy
      action= policy[state]
      new_state= np.random.choice(list(set(state_list)-set([state])))
      rewards=env_data.values[state][action]
      V[state]= (rewards + gamma*(old_V))/9999
      biggest_change= max(biggest_change, abs(V[state] - old_V))
      deltas.append(biggest_change)
      if biggest_change < small_change:
        break

In [48]:
V[state]

0.34608061782442456